![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [52]:
import pandas as pd
import numpy as np

# Start coding here...

main_frame = pd.read_csv('bank_marketing.csv')

                #clients table

client = pd.DataFrame()
client['id'] = main_frame['client_id']
client['age'] = main_frame['age']
client['job'] = main_frame['job'].apply(lambda x: x.strip('.'))
client['marital'] = main_frame['marital']
client['education'] = main_frame['education']
client['credit_default'] = main_frame['credit_default']
client['housing'] = main_frame['housing']
client['loan'] = main_frame['loan']

client['education'] = client['education'].apply(lambda x: x.replace('.','_'))
client['education'] = client['education'].replace('unknown', np.NaN)

client['credit_default'] = client['credit_default'].replace('yes','1')
client['credit_default'] = client['credit_default'].replace('no','0')
client['credit_default'] = client['credit_default'].replace('unknown', np.NaN)
client['credit_default'] = client['credit_default'].astype(float)

client['housing'] = client['housing'].replace('yes','1')
client['housing'] = client['housing'].replace('no','0')
client['housing'] = client['housing'].replace('unknown', np.NaN)
client['housing'] = pd.to_numeric(client['housing'])

client['loan'] = client['loan'].replace('yes','1')
client['loan'] = client['loan'].replace('no','0')
client['loan'] = client['loan'].replace('unknown', np.NaN)
client['loan'] = pd.to_numeric(client['loan'])

                #campaign table

campaign = pd.DataFrame()
campaign['campaign_id'] = 1
campaign['client_id'] = main_frame['client_id']
campaign['number_contacts']= main_frame['campaign']
campaign['contact_duration'] = main_frame['duration']
campaign['pdays'] = main_frame['pdays']
campaign['previous_campaign_contacts'] = main_frame['previous']
campaign['previous_outcome'] = main_frame['poutcome']
campaign['campaign_outcome'] = main_frame['y']
campaign['last_contact_date'] = '2022-' + main_frame['month'].apply(lambda x: x.capitalize()) + '-'+ main_frame['day'].apply(lambda x: str(x))

campaign['campaign_outcome'] = campaign['campaign_outcome'].replace('yes','1')
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace('no','0')

campaign['previous_outcome'] = campaign['previous_outcome'].replace('success','1')
campaign['previous_outcome'] = campaign['previous_outcome'].replace('failure','0')

campaign['previous_outcome'] = campaign['previous_outcome'].replace('nonexistent', np.NaN)

campaign['previous_outcome'] = pd.to_numeric(campaign['previous_outcome'])
campaign['campaign_outcome'] = pd.to_numeric(campaign['campaign_outcome'])

campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'])


                #economics table

economics = pd.DataFrame()
economics['client_id'] = main_frame['client_id']
economics['emp_var_rate'] = main_frame['emp_var_rate']
economics['cons_price_idx'] = main_frame['cons_price_idx']
economics['euribor_three_months'] = main_frame['euribor3m']
economics['number_employed'] = main_frame['nr_employed']

            #csvs and sql snippets
client.to_csv('client.csv', sep = ',', index = False)
campaign.to_csv('campaign.csv', sep = ',', index = False)
economics.to_csv('economics.csv', sep=',', index = False)

client_table = '''
CREATE TABLE client (
  id SERIAL PRIMARY KEY,
  age INT,
  job TEXT,
  marital TEXT,
  education TEXT,
  credit_default BOOLEAN,
  housing BOOLEAN,
  loan BOOLEAN
  );
  
  \copy client from 'client.csv' DELIMITER ',' CSV HEADER
'''

campaign_table = '''
CREATE TABLE campaign (
	campaign_id SERIAL PRIMARY KEY,
  	client_id SERIAL REFERENCES client (id),
  	number_contacts INT,
  	contact_duration INT,
  	pdays INT,
  	previous_campaign_contacts INT,
  	previous_outcome BOOLEAN,
  	campaign_outcome BOOLEAN,
  	last_contact_date DATE
  );
  
  \copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
'''

economics_table = '''
CREATE TABLE economics (
  	client_id SERIAL REFERENCES client (id),
  	emp_var_rate FLOAT,
  	cons_price_idx FLOAT,
  	euribor_three_months FLOAT,
  	number_employed FLOAT
  );

\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
'''